In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from scipy import ndimage
# from scipy.optimize import curve_fit
from scipy.optimize import minimize
# from scipy.stats import median_absolute_deviation
import xarray as xr

import debrisglobal.globaldebris_input as debris_prms
import class_climate_debris


# debug=True

# Degree-day factor of clean ice (m w.e. d-1 degC-1)
output_fp = debris_prms.output_fp + 'mb_grad_model/'
if not os.path.exists(output_fp):
    os.makedirs(output_fp)

In [2]:
def date_yrfrac_to_yyyymm(date_yrfrac):
    if int(date_yrfrac)%4 == 0:
        doy = date_yrfrac%1*366
    else:
        doy = date_yrfrac%1*365
    return datetime.strptime(str(int(date_yrfrac)) + '-' + str(int(doy)),'%Y-%j').date().strftime('%Y-%m')

def create_datestable(startdate, enddate):
    """
    Create table of year, month, day, water year, season and number of days in the month.

    Parameters
    ----------
    startdate, enddate : str
        start and end data in string format ('YYYY-MM')

    Returns
    -------
    dates_table : pd.DataFrame
        table where each row is a timestep and each column is attributes (date, year) of that timestep
    """
    # Generate dates_table using date_range function
    # Automatically generate dates from start date to end data using a monthly frequency (MS), which generates
    # monthly data using the 1st of each month
    dates_table = pd.DataFrame({'date' : pd.date_range(startdate, enddate, freq='MS')})
    # Select attributes of DateTimeIndex (dt.year, dt.month, and dt.daysinmonth)
    dates_table['year'] = dates_table['date'].dt.year
    dates_table['month'] = dates_table['date'].dt.month
    dates_table['daysinmonth'] = dates_table['date'].dt.daysinmonth
    dates_table['timestep'] = np.arange(len(dates_table['date']))
    # Set date as index
    dates_table.set_index('timestep', inplace=True)
    return dates_table


def annualweightedmean_array(var, dates_table):
    """ Calculate annual mean of variable according to the timestep """        
    dayspermonth = dates_table['daysinmonth'].values.reshape(-1,12)
    #  creates matrix (rows-years, columns-months) of the number of days per month
    daysperyear = dayspermonth.sum(axis=1)
    #  creates an array of the days per year (includes leap years)
    weights = (dayspermonth / daysperyear[:,np.newaxis]).reshape(-1)
    #  computes weights for each element, then reshapes it from matrix (rows-years, columns-months) to an array, 
    #  where each column (each monthly timestep) is the weight given to that specific month
    var_annual = (var*weights[np.newaxis,:]).reshape(-1,12).sum(axis=1).reshape(-1,daysperyear.shape[0])
    #  computes matrix (rows - bins, columns - year) of weighted average for each year
    #  explanation: var*weights[np.newaxis,:] multiplies each element by its corresponding weight; .reshape(-1,12) 
    #    reshapes the matrix to only have 12 columns (1 year), so the size is (rows*cols/12, 12); .sum(axis=1) 
    #    takes the sum of each year; .reshape(-1,daysperyear.shape[0]) reshapes the matrix back to the proper 
    #    structure (rows - bins, columns - year)
    # If averaging a single year, then reshape so it returns a 1d array
    if var_annual.shape[1] == 1:
        var_annual = var_annual.reshape(var_annual.shape[0])
    return var_annual


def calc_mb_mwea(bin_elev, glacier_gcm_prec, glacier_gcm_temp, glacier_gcm_lr, glacier_gcm_elev, dates_table, 
                 prec_factor=1, temp_change=0, ddf_ice=0.006, ddf_snow=None):
    """ Climatic mass balance (accumulation + refreeze - melt) according to PyGEM """
    
    # Degree-day factor
    if ddf_snow is not None:
        ddf = ddf_snow
    else:
        ddf = ddf_ice
    
    # Temperature based on lapse rate
    bin_gcm_temp = glacier_gcm_temp + glacier_gcm_lr * (bin_elev - glacier_gcm_elev) + temp_change
    
    # Accumulation
    bin_snow = prec_factor * glacier_gcm_prec.copy()
    bin_snow[bin_gcm_temp > debris_prms.Tsnow_threshold] = 0
    bin_snow[bin_snow < debris_prms.snow_min] = 0
    bin_acc_mwea = bin_snow.sum() / 12
    
    # Melt (clean ice)
    # Monthly temperature superimposed with daily temperature variability
    # daily temperature variation in each bin for the monthly timestep
    melt_total = 0
    np.random.seed(0)
    for step, dayspermonth in enumerate(dates_table['daysinmonth'].values):
        bin_tempstd_daily = (np.random.normal(loc=0, scale=glacier_gcm_tempstd[step], size=dayspermonth)
                             .reshape(1,dayspermonth))
        # daily temperature in each bin for the monthly timestep
        bin_temp_daily = bin_gcm_temp[step] + bin_tempstd_daily
        # remove negative values
        bin_temp_daily[bin_temp_daily < 0] = 0
        # Melt
        melt_month = bin_temp_daily.sum() * ddf_ice
        melt_total += melt_month
    bin_melt_mwea = melt_total / 12    
    
    # Refreeze based on annual air temperature (Woodward etal. 1997)
    #  R(m) = (-0.69 * Tair + 0.0096) * 1 m / 100 cm
    if dates_table.shape[0]%12 > 0:
        dates_table_cropped = dates_table.loc[0:dates_table.shape[0]-1-dates_table.shape[0]%12,:]
        bin_gcm_temp_cropped = bin_gcm_temp[0:dates_table.shape[0]-dates_table.shape[0]%12]
    else:
        dates_table_cropped = dates_table
        bin_gcm_temp_cropped = bin_gcm_temp
    bin_temp_annual = annualweightedmean_array(bin_gcm_temp_cropped, dates_table_cropped)
    bin_refreezepotential_annual = (-0.69 * bin_temp_annual + 0.0096) * 1/100
    # Remove negative refreezing values
    bin_refreezepotential_annual[bin_refreezepotential_annual < 0] = 0
    bin_refreeze_mwea = bin_refreezepotential_annual.mean()
    if bin_refreeze_mwea > bin_melt_mwea:
        bin_refreeze_mwea = bin_melt_mwea
        
    # Mass balance (mwea)
    bin_mb_mwea = bin_acc_mwea + bin_refreeze_mwea - bin_melt_mwea 
    
    return bin_mb_mwea


def calc_mbclim_ela(elev_bins, zmin_mb_mwea, zmax_mb_mwea, z_ela, mf_values):
    mbclim_grad_mwea = np.zeros(elev_bins.shape)
    elev_ltzmed_idx = np.where(elev_bins < z_ela)[0]
    mbclim_grad_mwea[elev_ltzmed_idx] = (
        -1*zmin_mb_mwea * (elev_bins[elev_ltzmed_idx] - z_ela) / (z_ela - zmin))
    elev_gtzmed_idx = np.where(elev_bins > z_ela)[0]
    mbclim_grad_mwea[elev_gtzmed_idx] = zmax_mb_mwea * (elev_bins[elev_gtzmed_idx] - z_ela) / (zmax - z_ela)
    return mbclim_grad_mwea


def weighted_percentile(sorted_list, weights, percentile):
    """
    Calculate weighted percentile of a sorted list
    """
    weights_cumsum_norm_high = np.cumsum(weights) / np.sum(weights)
#     print(weights_cumsum_norm_high)
    weights_norm = weights / np.sum(weights)
    weights_cumsum_norm_low = weights_cumsum_norm_high - weights_norm
#     print(weights_cumsum_norm_low)
    
    percentile_idx_high = np.where(weights_cumsum_norm_high >= percentile)[0][0]
#     print(percentile_idx_high)
    percentile_idx_low = np.where(weights_cumsum_norm_low <= percentile)[0][-1]
#     print(percentile_idx_low)
    
    if percentile_idx_low == percentile_idx_high:
        value_percentile = sorted_list[percentile_idx_low]
    else:
        value_percentile = np.mean([sorted_list[percentile_idx_low], sorted_list[percentile_idx_high]])

    return value_percentile

In [3]:
# Glaciers optimized
overwrite = False
# rois = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15', '16','17','18']
rois = ['13','14','15']

data_source = 'regional'
# data_source = 'individual_glaciers'

if data_source in ['individual_glaciers']:
    mb_shean_fullfn = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/Shean_2019_0213/' + 
                       'hma_mb_20190215_0815_std+mean_all_filled_bolch.csv')
    mb_shean_df = pd.read_csv(mb_shean_fullfn)
    mb_shean_df['RGIId'] = ['RGI60-' + str(int(x)) + '.' + str(int(np.round((x - int(x)) * 1e5,0))).zfill(5) 
                            for x in mb_shean_df.RGIId.values]
elif data_source in ['regional']:
    roi_mbobs_dict = {'01': [-0.85, 0.19],
                      '02': [-0.83, 0.40],
                      '03': [-0.57, 0.80],
                      '04': [-0.57, 0.70],
                      '05': [-0.63, 0.21],
                      '06': [-0.71, 0.43],
                      '07': [-0.47, 0.23],
                      '08': [-0.49, 0.27],
                      '09': [-0.47, 0.37],
                      '10': [-0.37, 0.31],
                      '11': [-0.87, 0.07],
                      '12': [-0.90, 0.57],
                      '13': [-0.19, 0.15],
                      '14': [-0.11, 0.15],
                      '15': [-0.44, 0.15],
                      '16': [-1.03, 0.83],
                      '17': [-1.18, 0.38],
                      '18': [-0.68, 1.15]}


reg_output_cns = ['roi', 'area_km2', 'mb_obs_mwea', 'mb_obs_mwea_std', 'mb_mod_mwea', 'mb_mod_mwea_clean']
reg_output_df = pd.DataFrame(np.zeros((len(rois),len(reg_output_cns))), columns=reg_output_cns)
for nroi, roi in enumerate(rois):
    
    output_fn = roi + '_mb_grad_model_opt.csv'
    
    if overwrite or not os.path.exists(output_fp + output_fn):
    
        if roi in ['13','14','15']:
            roi_4dict = 'HMA'
        else:
            roi_4dict = roi

        rgiids = []
        mb_bin_fns_whd = []
        mb_bin_fns = []
        # Filepaths
        mb_bin_fp = debris_prms.output_fp + 'mb_bins/csv/_wdebris_hdts/'
        mb_bin_fp_extrap = debris_prms.output_fp + 'mb_bins/csv/_wdebris_hdts_extrap/'
        mb_bin_fp_all = debris_prms.output_fp + 'mb_bins_all/csv/' + roi_4dict + '/'
        mb_bin_fp_all_nodhdt = debris_prms.output_fp + 'mb_bins_all/csv/' + roi_4dict + '/no_dhdt/'
        
        # Filepaths
        mb_bin_fp = debris_prms.output_fp + 'mb_bins/csv/_wdebris_hdts/'
        mb_bin_fp_extrap = debris_prms.output_fp + 'mb_bins/csv/_wdebris_hdts_extrap/'

        # Glaciers optimized
        mb_bin_fullfns = []
        for i in os.listdir(mb_bin_fp):
            if i.endswith('.csv'):
                reg_str = str(int(i.split('.')[0])).zfill(2)
                if reg_str == roi:
                    mb_bin_fns_whd.append(mb_bin_fp + i)
                    rgiids.append(i.split('_')[0])

        # Glaciers extrapolated
        for i in os.listdir(mb_bin_fp_extrap):
            if i.endswith('_extrap.csv'):
                reg_str = str(int(i.split('.')[0])).zfill(2)
                if reg_str == roi:
                    mb_bin_fns_whd.append(mb_bin_fp_extrap + i)
                    rgiids.append(i.split('_')[0])

        # Sorted files        
        mb_bin_fns_whd = [x for _,x in sorted(zip(rgiids, mb_bin_fns_whd))]
        rgiids = sorted(rgiids)     
        
        # Sorted files        
        rgiids = sorted(rgiids)  
        
        rgiids_wdata = []
        for nrgiid, rgiid in enumerate(rgiids):
            mb_bin_fn = rgiid + '_mb_bins.csv'
            mb_bin_fn_nodhdt = rgiid + '_bins.csv'
            if os.path.exists(mb_bin_fp_all + mb_bin_fn):
                mb_bin_fns.append(mb_bin_fp_all + mb_bin_fn)
                rgiids_wdata.append(rgiid)
            elif os.path.exists(mb_bin_fp_all_nodhdt + mb_bin_fn_nodhdt):
                mb_bin_fns.append(mb_bin_fp_all_nodhdt + mb_bin_fn_nodhdt)
                rgiids_wdata.append(rgiid)
            else:
                print(rgiid)

        assert len(mb_bin_fns_whd) == len(rgiids_wdata), 'mismatch between mb_bin_fns and rgiids_wdata'
        
        
        # ===== LOAD GLACIER DATA =====
        main_glac_rgi = debris_prms.selectglaciersrgitable(rgiids_wdata)
        main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
        main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
            360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
        main_glac_rgi['mb_bin_fullfn'] = mb_bin_fns
        main_glac_rgi['mb_bin_fullfn_whd'] = mb_bin_fns_whd
        
        # Add mass balance data
        if data_source in ['individual_glaciers']:
            shean_rgiid_idx = [i for i, item in enumerate(list(mb_shean_df.RGIId.values)) 
                               if item in list(main_glac_rgi.RGIId.values)]
            main_glac_rgi['mb_obs_mwea'] = mb_shean_df.loc[shean_rgiid_idx,'mb_mwea'].values
            main_glac_rgi['mb_obs_mwea_std'] = mb_shean_df.loc[shean_rgiid_idx,'mb_mwea_sigma'].values
        elif data_source in ['regional']:
            main_glac_rgi['mb_obs_mwea'] = roi_mbobs_dict[roi][0]
            main_glac_rgi['mb_obs_mwea_std'] = roi_mbobs_dict[roi][1]
            
        # ===== LOAD CLIMATE DATA =====
        # Dates
        if data_source in ['individual_glaciers']:
            start_date = date_yrfrac_to_yyyymm(debris_prms.mb_yrfrac_dict[roi_4dict][0])
            end_date = date_yrfrac_to_yyyymm(debris_prms.mb_yrfrac_dict[roi_4dict][1])
            dates_table = create_datestable(start_date, end_date)
        elif data_source in ['regional']:
            if roi in ['13','14','15']:
                dates_table = create_datestable('2000-06', '2018-05')
            else:    
                dates_table = create_datestable('2006-06', '2015-05')

        # GCM
        gcm = class_climate_debris.GCM(name='ERA5')
        # Air temperature [degC]
        gcm_temp, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.temp_fn, gcm.temp_vn, main_glac_rgi,
                                                                     dates_table)
        gcm_tempstd, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.tempstd_fn, gcm.tempstd_vn,
                                                                        main_glac_rgi, dates_table)
        # Precipitation [m]
        gcm_prec, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.prec_fn, gcm.prec_vn, main_glac_rgi,
                                                                     dates_table)
        # Elevation [m asl]
        gcm_elev = gcm.importGCMfxnearestneighbor_xarray(gcm.elev_fn, gcm.elev_vn, main_glac_rgi)
        # Lapse rate
        gcm_lr, gcm_dates = gcm.importGCMvarnearestneighbor_xarray(gcm.lr_fn, gcm.lr_vn, main_glac_rgi, dates_table)
        
        
        # ===== LOOP THROUGH THE GLACIERS ============================================================================
        output_cns = ['rgiid', 'area_km2', 'mb_obs_mwea', 'mb_mod_mwea', 'mb_mod_mwea_clean', 'zela', 'zmed', 
                      'zmin', 'z25', 'z75', 'zmax', 'zmin_mb_mwea', 'zmax_mb_mwea', 'ela_opt_percentile', 'ddfice',
                      'precfactor', 'tempchange']
        output_df = pd.DataFrame(np.zeros((main_glac_rgi.shape[0], len(output_cns))), columns=output_cns)

#     for batman in [0]:
#         debug = True
#         for nglac, glac_idx in enumerate(main_glac_rgi.index.values[36:37]):
        debug = False
        for nglac, glac_idx in enumerate(main_glac_rgi.index.values): 

            glac_str = main_glac_rgi.loc[glac_idx,'rgino_str']
            rgiid = main_glac_rgi.loc[glac_idx,'RGIId']
            region = glac_str.split('.')[0]

            if int(region) < 10:
                glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
            else:
                glac_str_noleadzero = glac_str

            if nglac%200 == 0:
#             if nglac%1 == 0:
                print(nglac, glac_str)

            # Select climate data
            glacier_gcm_elev = gcm_elev[glac_idx]
            glacier_gcm_prec = gcm_prec[glac_idx,:]
            glacier_gcm_temp = gcm_temp[glac_idx,:]
            glacier_gcm_tempstd = gcm_tempstd[glac_idx,:]
            glacier_gcm_lr = gcm_lr[glac_idx,:]

            # Load bins
            mb_df = pd.read_csv(main_glac_rgi.loc[glac_idx,'mb_bin_fullfn'])
            mb_df.loc[:,:] = mb_df.values.astype(np.float64)
            mb_df_whd = pd.read_csv(main_glac_rgi.loc[glac_idx,'mb_bin_fullfn_whd'])
            mb_df_whd.loc[:,:] = mb_df_whd.values.astype(np.float64)
            # Weighted melt factor according to debris-covered area in the bin
            mf_values = mb_df['dc_bin_area_perc']/100 * mb_df_whd['mf_ts_mean'] + (100-mb_df['dc_bin_area_perc'])/100

            # Min, max, median elevations
            elev_bins = mb_df.bin_center_elev_m.values
            area_bins = mb_df.z1_bin_area_valid_km2.values
            zmin = elev_bins.min()
            zmax = elev_bins.max()
            zmed = weighted_percentile(elev_bins, area_bins, 0.5)
            z25 = weighted_percentile(elev_bins, area_bins, 0.25)
            z75 = weighted_percentile(elev_bins, area_bins, 0.75)

            # Calibrate the ELA against the observation
            mb_obs_mwea = main_glac_rgi.loc[glac_idx, 'mb_obs_mwea']
            mb_obs_mwea_std = main_glac_rgi.loc[glac_idx, 'mb_obs_mwea_std']

            if debug:
                print('mb_obs:', np.round(mb_obs_mwea,2))
                
            # Bounds
            ddfice = 0.006
            ddfice_step = 0.005
            ddfice_bndlow, ddfice_bndhigh = 0.0035, 0.009
            precfactor = 1.5
            precfactor_step = 0.25
            precfactor_bndlow, precfactor_bndhigh = 0.8, 2
            tc = 0
            tc_step = 0.05
            tc_bndlow, tc_bndhigh = -10, 10
            # Boundaries and steps
            zstep = 10
            dif_mb_mwea_threshold = 0.01
            zlimit_lower = z25
            if zlimit_lower == zmin:
                zlimit_lower = zmin + 10
            continue_while_loop = True
            nround = 0
            dif_mb_mwea = 1e6
            abs_dif_mb_mwea_opt = 1e6
            ela_perc_thresholds = [40, 85]
            while continue_while_loop:
#                 if debug:
#                     print('\nRound', nround)
                # Maximum accumulation (mwea)
                ddfsnow = ddfice / 1.5
                zmax_mb_mwea = calc_mb_mwea(zmax, glacier_gcm_prec, glacier_gcm_temp, glacier_gcm_lr, 
                                            glacier_gcm_elev, dates_table, prec_factor=precfactor, 
                                            temp_change=tc, ddf_snow=ddfsnow)
            
                # Maximum melt (mwea)
                zmin_mb_mwea = calc_mb_mwea(zmin, glacier_gcm_prec, glacier_gcm_temp, glacier_gcm_lr, 
                                            glacier_gcm_elev, dates_table, prec_factor=precfactor, 
                                            temp_change=tc, ddf_ice=ddfice)
    
                if debug:
                    print('zmax mb:', np.round(zmax_mb_mwea,2), 'zmin_mb:', np.round(zmin_mb_mwea,2))
                
                # Calibrate ELA
                if nround == 0:
                    z_list = list(np.arange(zlimit_lower, zmax+zstep/2, zstep))
                for z_ela in z_list:
                    # Mass balance gradient model
                    mbclim_grad_mwea = calc_mbclim_ela(elev_bins, zmin_mb_mwea, zmax_mb_mwea, z_ela, mf_values)
                    mbclim_grad_mwea_wmf = mbclim_grad_mwea * mf_values

                    # Glacier-wide mass balance
                    glac_mb_mwea = (mbclim_grad_mwea_wmf * area_bins).sum() / area_bins.sum()
                    glac_mb_mwea_clean = (mbclim_grad_mwea * area_bins).sum() / area_bins.sum()
                    dif_mb_mwea = glac_mb_mwea - mb_obs_mwea

                    
                    if nround > 0 and debug:
                        print('z_ela:', z_ela, 'prms:', ddfice, precfactor, tc, 
                              'mb opt/mod/obs/dif:', np.round(glac_mb_mwea_opt,2), np.round(glac_mb_mwea,2), 
                              np.round(mb_obs_mwea,2), np.round(dif_mb_mwea,2))
                
#                 continue_while_loop = False

                    # Get best value
                    if abs(dif_mb_mwea) < abs_dif_mb_mwea_opt:
                        z_ela_opt = z_ela
                        ddfice_opt = ddfice
                        pf_opt = precfactor
                        tc_opt = tc
                        glac_mb_mwea_opt = glac_mb_mwea
                        glac_mb_mwea_clean_opt = glac_mb_mwea_clean
                        dif_mb_mwea_opt = dif_mb_mwea
                        abs_dif_mb_mwea_opt = abs(dif_mb_mwea)
                        

                    # If jump over the best value, then stop optimization and go with the minimum
                    if -1*np.sign(dif_mb_mwea_opt) == np.sign(dif_mb_mwea):
                        continue_while_loop = False

                # Get upper and lower limits
                if nround == 0:
                    if z_ela_opt < zmed:
                        z_list = list(np.arange(z25, zmed+zstep/2, zstep))
                    else:
                        z_list = list(np.arange(zmed, zmax+zstep/2, zstep))
                    
    #             if debug:
    #                 print('\n',z_ela_opt, ddfice, precfactor, tc, glac_mb_mwea_opt, mb_obs_mwea, abs_dif_mb_mwea_opt)

                # ELA percentile
                lt_ela_idx = np.where(elev_bins <= z_ela_opt)[0]
                ela_opt_percentile = area_bins[lt_ela_idx].sum() / area_bins.sum() * 100
    
                # If optimized, then done
                if (abs(dif_mb_mwea_opt) <= dif_mb_mwea_threshold and 
                    (ela_opt_percentile < ela_perc_thresholds[0] or ela_opt_percentile > ela_perc_thresholds[1])):
                    continue_while_loop = False
                # Otherwise, iterate through various ddfice, precfactors, tempchange
                else:
                    if dif_mb_mwea_opt > 0:
                        # PyGEM initial calibration
                        # Decrease kp, increase Tbias
                        precfactor = precfactor - precfactor_step
                        if precfactor < precfactor_bndlow:
                            precfactor = precfactor_bndlow
                            ddfice = ddfice + ddfice_step
                            if ddfice > ddfice_bndhigh:
                                ddfice = ddfice_bndhigh
                                tc = tc + tc_step
                        
                        # Initial method
#                         ddfice = ddfice + ddfice_step
#                         if ddfice >= ddfice_bndhigh:
#                             ddfice = ddfice_bndhigh
#                             precfactor = precfactor - precfactor_step
#                             if precfactor <= precfactor_bndlow:
#                                 precfactor = precfactor_bndlow
#                                 tc = tc + tc_step
                    elif dif_mb_mwea_opt < 0:
                        precfactor = precfactor + precfactor_step
                        if precfactor > precfactor_bndhigh:
                            precfactor = precfactor_bndhigh
                            ddfice = ddfice - ddfice_step
                            if ddfice < ddfice_bndlow:
                                ddfice = ddfice_bndlow
                                tc = tc - tc_step
#                         ddfice = ddfice - ddfice_step
#                         if ddfice <= ddfice_bndlow:
#                             ddfice = ddfice_bndlow
#                             precfactor = precfactor + precfactor_step
#                             if precfactor >= precfactor_bndhigh:
#                                 precfactor = precfactor_bndhigh
#                                 tc = tc - tc_step

                    if precfactor > precfactor_bndhigh:
                        precfactor = precfactor_bndhigh
                    if precfactor < precfactor_bndlow:
                        precfactor = precfactor_bndlow
                    if tc > tc_bndhigh:
                        tc = tc_bndhigh
                        print(nglac, glac_str, 'HERE - hit upper bound')
                        continue_while_loop = False
                    if tc < tc_bndlow:
                        tc = tc_bndlow
                        print(nglac, glac_str, 'HERE - hit lower bound')
                        continue_while_loop = False

                nround += 1

            if debug:
                print('\n',z_ela_opt, ddfice_opt, pf_opt, tc_opt, glac_mb_mwea_opt)

            
            if debug:
                print('  mb_obs (mwea):', np.round(mb_obs_mwea,2), '+/-', np.round(mb_obs_mwea_std,2))
                print('  z_ela opt:', np.round(z_ela_opt), '(' + str(np.round(ela_opt_percentile,2)) + '%)', 
                      'z25, zmed, z75:', z25, zmed, z75)
                print('  mb_mod (mwea):', np.round(glac_mb_mwea_opt,2), 'mb_mod (clean):', 
                      np.round(glac_mb_mwea_clean_opt,2))
            
            
            if dif_mb_mwea_opt > mb_obs_mwea_std:
                print('  ' + glac_str + ' BAD FIT')
                error_fp = output_fp + 'errors/poor_fit/' + roi + '/'
                if not os.path.exists(error_fp):
                    os.makedirs(error_fp)
                with open(error_fp + glac_str + "-difference_gt_zempstd.txt", "w") as text_file:
                    text_file.write(glac_str + ' modeled mass balance difference off by more than zempstd')
            if z_ela_opt == z25:
#                 print('  ' + glac_str + ' HAS ISSUE - z25')
                error_fp = output_fp + 'errors/z25/' + roi + '/'
                if not os.path.exists(error_fp):
                    os.makedirs(error_fp)
                with open(error_fp + glac_str + "-poor_mbgrad_model-z25.txt", "w") as text_file:
                    text_file.write(glac_str + ' bad fit for mb grad model; z_ela_opt equals z25')
            if z_ela_opt == zmax:
#                 print('  ' + glac_str + ' HAS ISSUE - zmax')
                error_fp = output_fp + 'errors/zmax/' + roi + '/'
                if not os.path.exists(error_fp):
                    os.makedirs(error_fp)
                with open(error_fp + glac_str + "-poor_mbgrad_model-zmax.txt", "w") as text_file:
                    text_file.write(glac_str + ' bad fit for mb grad model; z_ela_opt equals zmax')

            output_df.loc[glac_idx,:] = [glac_str, area_bins.sum(), mb_obs_mwea, glac_mb_mwea_opt, 
                                         glac_mb_mwea_clean_opt, z_ela_opt, zmed, zmin, z25, z75, zmax, 
                                         zmin_mb_mwea, zmax_mb_mwea, ela_opt_percentile, ddfice, precfactor, tc]
        output_df.to_csv(output_fp + output_fn, index=False)

    else:
        output_df = pd.read_csv(output_fp + output_fn)
    
    # Ensure RGIIds are correctly formatted
    output_df['rgiid'] = ['RGI60-' + str(x).split('.')[0].zfill(2) + '.' + 
                          str(int(np.round((np.float(x) - int(np.float(x))) * 1e5,0))).zfill(5) 
                          for x in output_df.rgiid.values]

    main_glac_rgi_all = debris_prms.selectglaciersrgitable(rgi_regionsO1=[roi], rgi_regionsO2='all',
                                                           rgi_glac_number='all')
    
    # Add the observations for the glaciers not modeled
    if data_source in ['individual_glaciers']:
        # Calculate the change in specific mass balance over the region
        mb_shean_df['roi'] = [x.split('-')[1].split('.')[0] for x in mb_shean_df.RGIId.values]
        mb_shean_df_roi = mb_shean_df[mb_shean_df.roi == roi]
        mb_shean_df_roi.reset_index(inplace=True, drop=True)
        reg_mb_obs_mwea = (mb_shean_df_roi.mb_mwea * mb_shean_df_roi.area_m2).sum() / mb_shean_df_roi.area_m2.sum()
        reg_mb_obs_mwea_std = 0.15
        
        # Dictionary of glacier-wide mass balance for every glacier
        mb_mwea_debris_dict = dict(zip(mb_shean_df.RGIId.values, mb_shean_df.mb_mwea.values))
        mb_mwea_clean_dict = dict(zip(mb_shean_df.RGIId.values, mb_shean_df.mb_mwea.values))

    elif data_source in ['regional']:
        reg_mb_obs_mwea = roi_mbobs_dict[roi][0]
        reg_mb_obs_mwea_std = roi_mbobs_dict[roi][1]
        main_glac_rgi_all['mb_obs'] = reg_mb_obs_mwea
        mb_mwea_debris_dict = dict(zip(main_glac_rgi_all.RGGIId.values, main_glac_rgi_all.mb_obs.values))
        mb_mwea_clean_dict = dict(zip(main_glac_rgi_all.RGGIId.values, main_glac_rgi_all.mb_obs.values))
        
    # Update dictionaries for modeled debris-covered glaciers
    mb_mwea_debris_dict_output = dict(zip(output_df.rgiid.values, output_df.mb_mod_mwea.values))
    mb_mwea_clean_dict_output = dict(zip(output_df.rgiid.values, output_df.mb_mod_mwea_clean.values))
    
    mb_mwea_debris_dict.update(mb_mwea_debris_dict_output)
    mb_mwea_clean_dict.update(mb_mwea_clean_dict_output)
    
    main_glac_rgi_all['mb_mwea'] = main_glac_rgi_all.RGIId.map(mb_mwea_debris_dict)
    main_glac_rgi_all['mb_mwea_clean'] = main_glac_rgi_all.RGIId.map(mb_mwea_clean_dict)

    reg_mb_mwea_wdebris = (main_glac_rgi_all.Area * main_glac_rgi_all.mb_mwea).sum() / main_glac_rgi_all.Area.sum()
    reg_mb_mwea_nodebris = ((main_glac_rgi_all.Area * main_glac_rgi_all.mb_mwea_clean).sum() / 
                            main_glac_rgi_all.Area.sum())

    assert np.abs(reg_mb_mwea_wdebris - reg_mb_obs_mwea) < reg_mb_obs_mwea_std, 'Reg mass balance outside uncertainty'

    print('\nreg mb obs (mwea):', np.round(reg_mb_obs_mwea,2))
    print('reg mb mod (mwea), debris:', np.round(reg_mb_mwea_wdebris,2), 'vs clean:', np.round(reg_mb_mwea_nodebris,2),'\n')
    reg_output_df.loc[nroi,:] = [roi, main_glac_rgi_all.Area.sum(), reg_mb_obs_mwea, reg_mb_obs_mwea_std, 
                                 reg_mb_mwea_wdebris, reg_mb_mwea_nodebris]
    
# # Export regional data
# all_area = reg_output_df.area_km2.sum()
# all_zemp_mean = (reg_output_df.area_km2 * reg_output_df.zemp_mean).sum() / reg_output_df.area_km2.sum()
# all_zemp_std = 0.20
# all_mb_mwea = (reg_output_df.area_km2 * reg_output_df.mb_mwea).sum() / reg_output_df.area_km2.sum()
# all_mb_mwea_clean = (reg_output_df.area_km2 * reg_output_df.mb_mwea_clean).sum() / reg_output_df.area_km2.sum()

# all_df = pd.DataFrame(np.zeros((1,len(reg_output_cns))), columns=reg_output_cns)
# all_df.loc[0,:] = ['all', all_area, all_zemp_mean, all_zemp_std, all_mb_mwea, all_mb_mwea_clean]

# all_output_df = pd.concat([reg_output_df, all_df], axis=0)
# all_output_df['mb_clean_dif'] = all_output_df.mb_mwea - all_output_df.mb_mwea_clean
# all_output_df_fn = 'reg_mbgrad_impact.csv'
# all_output_df.to_csv(output_fp + all_output_df_fn, index=False)


8834 glaciers in region 13 are included in this model run: ['00067', '00080', '00093', '00137', '00175', '00188', '00190', '00198', '00199', '00200', '00202', '00203', '00209', '00210', '00211', '00217', '00218', '00221', '00222', '00223', '00226', '00228', '00233', '00235', '00236', '00240', '00246', '00248', '00250', '00299', '00335', '00345', '00358', '00386', '00391', '00394', '00399', '00400', '00402', '00413', '00420', '00500', '00502', '00503', '00505', '00507', '00508', '00515', '00516', '00517'] and more
This study is focusing on 8834 glaciers in region [13]


NameError: name 'roi_zemp_dict' is not defined

In [4]:
print('\n\nDONE!\n\n')



DONE!




In [5]:
# np.where(main_glac_rgi.rgino_str == '15.00058')

In [5]:
# Glaciers optimized
# rois = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15', '16','17','18']
rois = ['13','14','15']

for nroi, roi in enumerate(rois):
    
    output_fn = roi + '_mb_grad_model_opt.csv'
    output_df = pd.read_csv(output_fp + output_fn)
    output_df['rgiid'] = [str(x).split('.')[0].zfill(2) + '.' + str(x).split('.')[1] 
                          for x in output_df.rgiid.values]
    
    # ===== HISTOGRAM: ELA ======
    ax = output_df['ela_opt_percentile'].plot.hist(bins=20, alpha=0.5)
    fig = ax.get_figure()
    hist_fp = output_fp + 'hist/'
    if not os.path.exists(hist_fp):
        os.makedirs(hist_fp)
    fig.savefig(hist_fp + roi + '_ela_opt_percentile_hist.png')
    fig.clf()

<Figure size 432x288 with 0 Axes>